In [1]:
import numpy as np

# Generation of initial population of random antibodies

In [1113]:
# m: number of machines
# p: number of parts
# population_size : number of antibody of the initial population
def generation_initial_population(p, m, population_size):
    MaxCell = min(p,m) #calculation of max number of cells
    number_of_zeros = MaxCell - 1 #number of zeros in each antibody
    antibodies = np.empty(shape=(population_size, p+m+number_of_zeros), dtype=int)
    antibody = np.append(np.array([*range(1,p+m+1)]), np.zeros(number_of_zeros,dtype=int))
    for i in range(0,population_size):
        np.random.shuffle(antibody) #random positions in the array
        antibodies[i] = antibody
    return antibodies

In [1125]:
generation_initial_population(p = 4, m = 8, population_size = 3)

array([[ 0,  2,  3, 12, 10,  6,  7,  8,  5,  0, 11,  9,  1,  4,  0],
       [ 1, 12,  9,  0,  3,  7,  8,  2,  4,  6, 10, 11,  5,  0,  0],
       [ 3,  0,  2,  9, 10, 12, 11,  0,  1,  4,  6,  7,  5,  8,  0]])

# Evaluate all existing antibodies and compute their affinities

traducción del formato del dataset a matriz trabajo-estación

# Select N% of antibodies with highest affinities

# Clone the selected antibodies